<a href="https://colab.research.google.com/github/JandreiSS/analise-fundamentalista/blob/master/An%C3%A1lise_Fundamentalista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import string
import requests
import warnings
warnings.filterwarnings('ignore')

In [ ]:
url = 'http://fundamentus.com.br/resultado.php'

In [ ]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [ ]:
r = requests.get(url, headers=header)

In [ ]:
dflist = pd.read_html(r.text, thousands='.', decimal=',')[0]

In [ ]:
for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
  dflist[coluna] = dflist[coluna].str.replace('.', '')
  dflist[coluna] = dflist[coluna].str.replace(',', '.')
  dflist[coluna] = dflist[coluna].str.rstrip('%').astype('float') / 100

In [ ]:
dflist

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,CFLU4,1000.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0888,0.1072,1.10,0.1768,0.3215,0.00,6.035100e+07,0.06,0.0814
1,CLAN3,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,-0.0105,0.00,1.012240e+09,0.00,-0.6396
2,CLSC6,0.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.2288,0.00,1.551160e+09,0.00,0.0666
3,CSTB3,150.00,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.4085,0.2898,2.60,0.2240,0.2011,0.00,8.420670e+09,0.14,0.3191
4,CSTB4,147.69,0.00,0.00,0.000,0.0000,0.000,0.00,0.00,0.00,0.00,0.00,0.4085,0.2898,2.60,0.2240,0.2011,0.00,8.420670e+09,0.14,0.3191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882,CEPE6,20.00,1545.85,0.90,-18.326,0.0000,0.167,-9.71,71.38,-0.33,309.99,256.58,-0.2567,-0.0119,0.95,0.0027,0.0006,409.09,1.667000e+09,3.36,0.0863
883,CEPE3,24.53,1895.99,1.10,-22.477,0.0000,0.204,-11.91,87.55,-0.41,326.16,269.96,-0.2567,-0.0119,0.95,0.0027,0.0006,0.00,1.667000e+09,3.36,0.0863
884,CEPE5,27.00,2086.90,1.21,-24.740,0.0000,0.225,-13.11,96.37,-0.45,334.98,277.26,-0.2567,-0.0119,0.95,0.0027,0.0006,5386.70,1.667000e+09,3.36,0.0863
885,BRAP3,34.27,2114.74,1.13,0.000,0.0212,1.081,0.00,-682.88,0.00,-684.35,-684.35,0.0000,0.0000,0.00,0.0000,0.0005,490748.00,1.052240e+10,0.00,0.0000


#<strong>Analisando Dados</strong>

In [ ]:
df = dflist[dflist['Liq.2meses'] > 1000000]

<p><strong>Enterprise Value / Earnings Before Interest and Taxes</strong></p>
<p>(Valor do Negócio / Lucro Antes dos Juros e Taxas)</p>

In [ ]:
ranking = pd.DataFrame()
ranking['Position'] = range(1,151)
ranking['EV/EBIT'] = df[df['EV/EBIT'] > 0].sort_values(by=['EV/EBIT'])['Papel'][:150].values
ranking['ROIC'] = df.sort_values(by=['ROIC'], ascending=False)['Papel'][:150].values

In [ ]:
ranking

,Position,EV/EBIT,ROIC
0,1,PSSA3,WIZS3
1,2,SMLS3,PSSA3
2,3,ENAT3,TOTS3
3,4,WIZS3,ODPV3
4,5,VALE3,SMLS3
...,...,...,...
145,146,RADL3,JHSF3
146,147,LOGG3,CAML3
147,148,PTBL3,BRKM5
148,149,PNVL3,LOGG3


Cálculo utilizando o rankeamento pela Fórmula Mágica: Buscar o papel, pegando a posição que ele tem no EV/EBIT e somando a posição que ele tem no ROIC. Quanto menor, melhor

In [ ]:
auxA = ranking.pivot_table(columns='EV/EBIT', values='Position')
auxB = ranking.pivot_table(columns='ROIC', values='Position')
t = pd.concat([auxA, auxB])

In [ ]:
t

,AALR3,ABEV3,AGRO3,ALPA4,ALSO3,ALUP11,AMAR3,ANIM3,ARZZ3,B3SA3,BEEF3,BRDT3,BRFS3,BRKM5,BRML3,BRPR3,CAMB3,CAML3,CARD3,CCPR3,CCRO3,CEAB3,CESP6,CIEL3,CLSC4,CMIG3,CMIG4,COCE5,COGN3,CPFE3,CPLE3,CPLE6,CRFB3,CSAN3,CSMG3,CSNA3,CVCB3,CYRE3,DIRR3,DTEX3,...,SAPR3,SAPR4,SBSP3,SEER3,SGPS3,SHUL4,SLCE3,SMLS3,SMTO3,SULA11,SUZB3,TASA4,TEND3,TESA3,TGMA3,TIET11,TIET4,TIMP3,TOTS3,TRIS3,TRPL4,TUPY3,UGPA3,UNIP6,USIM3,USIM5,VALE3,VIVT3,VIVT4,VLID3,VULC3,VVAR3,WIZS3,WSON33,YDUQ3,EZTC3,HAPV3,LWSA3,MGLU3,WEGE3
Position,114.0,88.0,68.0,138.0,81.0,6.0,50.0,120.0,139.0,136.0,26.0,106.0,43.0,130.0,73.0,121.0,51.0,144.0,72.0,76.0,66.0,84.0,102.0,30.0,28.0,13.0,12.0,53.0,141.0,46.0,17.0,20.0,60.0,122.0,25.0,41.0,10.0,133.0,70.0,135.0,...,11.0,7.0,38.0,40.0,105.0,90.0,47.0,2.0,80.0,9.0,142.0,100.0,45.0,78.0,42.0,63.0,61.0,56.0,87.0,93.0,23.0,29.0,108.0,31.0,62.0,57.0,5.0,75.0,74.0,33.0,49.0,140.0,4.0,27.0,79.0,NaN,NaN,NaN,NaN,NaN
Position,141.0,15.0,84.0,20.0,131.0,29.0,52.0,115.0,30.0,34.0,9.0,98.0,70.0,148.0,127.0,NaN,57.0,147.0,46.0,116.0,42.0,103.0,136.0,35.0,124.0,66.0,67.0,96.0,NaN,37.0,62.0,63.0,50.0,114.0,41.0,56.0,40.0,130.0,122.0,126.0,...,12.0,14.0,36.0,72.0,NaN,90.0,73.0,5.0,88.0,102.0,145.0,87.0,25.0,139.0,8.0,23.0,24.0,78.0,3.0,53.0,51.0,60.0,106.0,77.0,128.0,129.0,18.0,104.0,105.0,92.0,65.0,110.0,1.0,28.0,17.0,117.0,71.0,99.0,107.0,11.0


In [ ]:
rank = t.dropna(axis=1).sum().sort_values()
rank[:15]

PSSA3      3.0
WIZS3      5.0
SMLS3      7.0
SAPR4     21.0
SAPR11    21.0
VALE3     23.0
SAPR3     23.0
LEVE3     31.0
EQTL3     33.0
MRFG3     35.0
ALUP11    35.0
BEEF3     35.0
CVCB3     50.0
TGMA3     50.0
ENAT3     52.0
dtype: float64